# Seldon V2 Non Kubernetes Pipeline Examples


 * Build if needed and place `seldon` binary in your path
    * run `make build-seldon` from operator folder and add bin folder to PATH
 * Run Seldon V2 `make deploy-local-triton` from top level folder

In [ ]:
!which seldon

## Model Chaining

In [ ]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml

In [ ]:
!seldon model load -f ./models/tfsimple1.yaml 
!seldon model load -f ./models/tfsimple2.yaml 

In [ ]:
!seldon model status --model-name tfsimple1 -w ModelAvailable | jq .
!seldon model status --model-name tfsimple2 -w ModelAvailable | jq .

In [ ]:
!seldon model infer -m tfsimple1 \
  '{"inputs":[{"name":"INPUT0","data": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]}]}' 

In [ ]:
!cat ./pipelines/tfsimples.yaml

In [ ]:
!seldon pipeline load -f ./pipelines/tfsimples.yaml

In [ ]:
!seldon pipeline status -p tfsimples -w PipelineReady| jq .

In [ ]:
!seldon pipeline infer -p tfsimples --inference-mode grpc \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq .

In [ ]:
!seldon pipeline unload -p tfsimples

In [ ]:
!seldon model unload --model-name tfsimple1
!seldon model unload --model-name tfsimple2

## Model Join

In [ ]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml
!cat ./models/tfsimple3.yaml

In [ ]:
!seldon model load -f ./models/tfsimple1.yaml 
!seldon model load -f ./models/tfsimple2.yaml 
!seldon model load -f ./models/tfsimple3.yaml 

In [ ]:
!seldon model status --model-name tfsimple1 -w ModelAvailable | jq .
!seldon model status --model-name tfsimple2 -w ModelAvailable | jq .
!seldon model status --model-name tfsimple3 -w ModelAvailable | jq .

In [ ]:
!cat ./pipelines/tfsimples-join.yaml

In [ ]:
!seldon pipeline load -f ./pipelines/tfsimples-join.yaml

In [ ]:
!seldon pipeline status -p join -w PipelineReady | jq .

In [ ]:
!seldon pipeline infer -p join --inference-mode grpc \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq .

In [ ]:
!seldon pipeline unload -p join

In [ ]:
!seldon model unload --model-name tfsimple1
!seldon model unload --model-name tfsimple2
!seldon model unload --model-name tfsimple3